In [1]:
from skshapes.morphing import ElasticMetric, RigidMotion
from skshapes.loss import OptimalTransportLoss, LandmarkLoss, NearestNeighborsLoss
from skshapes.data import read
from skshapes.tasks import DistanceMatrix, Registration
import torch

In [3]:
import os

targetfolder = "data/SCAPE_low_resolution_aligned"
datafolder = "data/SCAPE_low_resolution"

files = os.listdir(datafolder)
reference = read(datafolder + "/" + files[0])
rest = [read(datafolder + "/" + file) for file in files[1:]]

N = reference.points.shape[0]
reference.landmarks = torch.arange(N, dtype=torch.int64)
reference.to_pyvista().save(targetfolder + "/" + files[0])

for i, mesh in enumerate(rest):
    print("Aligning " + files[i + 1] + " to " + files[0] + "...")
    mesh.landmarks = torch.arange(N, dtype=torch.int64)

    r = Registration(
        model=RigidMotion(),
        loss=LandmarkLoss(p=2),
        verbose=0,
        n_iter=2,
        device="cpu",
    )

    morphed = r.fit_transform(source=mesh, target=reference)
    morphed.to_pyvista().save(targetfolder + "/" + files[i + 1])

Aligning mesh005.ply to mesh047.ply...
Aligning mesh001.ply to mesh047.ply...
Aligning mesh042.ply to mesh047.ply...
Aligning mesh027.ply to mesh047.ply...
Aligning mesh003.ply to mesh047.ply...
Aligning mesh023.ply to mesh047.ply...
Aligning mesh026.ply to mesh047.ply...
Aligning mesh022.ply to mesh047.ply...
Aligning mesh028.ply to mesh047.ply...
Aligning mesh033.ply to mesh047.ply...
Aligning mesh021.ply to mesh047.ply...
Aligning mesh044.ply to mesh047.ply...
Aligning mesh004.ply to mesh047.ply...
Aligning mesh024.ply to mesh047.ply...
Aligning mesh070.ply to mesh047.ply...
Aligning mesh029.ply to mesh047.ply...
Aligning mesh034.ply to mesh047.ply...
Aligning mesh011.ply to mesh047.ply...
Aligning mesh048.ply to mesh047.ply...
Aligning mesh012.ply to mesh047.ply...
Aligning mesh068.ply to mesh047.ply...
Aligning mesh052.ply to mesh047.ply...
Aligning mesh000.ply to mesh047.ply...
Aligning mesh065.ply to mesh047.ply...
Aligning mesh018.ply to mesh047.ply...
Aligning mesh032.ply to m

In [4]:
import os

targetfolder = "data/SCAPE_low_resolution_aligned_ICP"
datafolder = "data/SCAPE_low_resolution"

files = os.listdir(datafolder)
reference = read(datafolder + "/" + files[0])
rest = [read(datafolder + "/" + file) for file in files[1:]]

reference.to_pyvista().save(targetfolder + "/" + files[0])

for i, mesh in enumerate(rest):
    print("Aligning " + files[i + 1] + " to " + files[0] + "...")

    r = Registration(
        model=RigidMotion(),
        loss=NearestNeighborsLoss(),
        verbose=0,
        n_iter=5,
        device="cuda:0",
    )

    morphed = r.fit_transform(source=mesh, target=reference)
    morphed.to_pyvista().save(targetfolder + "/" + files[i + 1])

Aligning mesh005.ply to mesh047.ply...
[KeOps] Generating code for formula ArgKMin_Reduction(Sum((Var(0,3,0)-Var(1,3,1))**2),0) ... OK
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at least one of the input tensors is not contiguous. Consider using contiguous data arrays to avoid unnecessary copies.
[pyKeOps] Warning : at le

In [25]:
from pykeops.torch import Vi, Vj

reference = torch.randn(100, 3)
target = torch.randn(200, 3)

X_i = Vi(reference)
X_j = Vj(target)

D_ij = ((X_i - X_j) ** 2).sum(-1)
correspondences = D_ij.argKmin(K=1, dim=1).view(-1)

torch.norm((reference - target[correspondences]), p=2, dim=1).mean()

tensor(0.4216)

In [7]:
# Load some shapes
import os

datafolder = "data/SCAPE_low_resolution_aligned"

source = read(datafolder + "/" + "mesh004.ply")
target = read(datafolder + "/" + "mesh007.ply")


shapes = [read(datafolder + "/" + f"mesh{i:03d}.ply") for i in range(10)]

r = Registration(
    model=ElasticMetric(n_steps=5),
    loss=OptimalTransportLoss(),
    verbose=1,
    n_iter=10,
    regularization=0,
)

r.fit(source=shapes[0], target=shapes[7])
newshape = r.transform(source=shapes[0])

Loss value at iteration 0 : 0.15649539232254028
Loss value at iteration 1 : 0.01606137678027153
Loss value at iteration 2 : 0.007564371917396784
Loss value at iteration 3 : 0.005975775420665741
Loss value at iteration 4 : 0.005975562613457441
Loss value at iteration 5 : 0.005975562147796154
Loss value at iteration 6 : 0.005975562147796154
Loss value at iteration 7 : 0.005975562147796154
Loss value at iteration 8 : 0.005975562147796154
Loss value at iteration 9 : 0.005975562147796154


In [6]:
newshape.to_pyvista().save("newshape.vtk")

In [19]:
from sklearn.cluster import SpectralClustering
import numpy as np

affinity_matrix = np.exp(-((distance_matrix / 0.001) ** 2))

clustering = SpectralClustering(affinity="precomputed", n_clusters=2).fit(
    affinity_matrix
)
print(clustering.labels_)

[0 0 0 0 0 0 1 0 1 0]


In [1]:
from skshapes.preprocessing import (
    LandmarkSetter,
    Pipeline,
    Decimation,
    AffineTransformation,
)
from skshapes.data import read
import os

# Read the shapes
datafolder = "/home/GitHub/scikit-shapes-draft/data/SCAPE/decimated/"
shapes = [read(datafolder + "/" + f"aligned_mesh{i:03d}.ply") for i in range(4)]

# Create a pipeline of preprocessing steps
preprocessing = Pipeline(
    steps=[
        LandmarkSetter(landmarks=[[0, 1], [2, 3], [4, 5], [6, 70]], by_indices=True),
        Decimation(target_reduction=0.95),
        AffineTransformation(matrix=[[2, 0, 0], [0, 2, 0], [0, 0, 2]]),
    ]
)

# Apply the preprocessing pipeline to the shapes and get the dataset
dataset = preprocessing.fit_transform(shapes=shapes)

for i in range(len(dataset)):
    print("Size of shape {} : {}".format(i, dataset[i].points.shape))
    print("Landmarks of shape {} : {}".format(i, dataset.landmarks[i]), end="\n\n")

Size of shape 0 : torch.Size([518, 3])
Landmarks of shape 0 : tensor([[ 0.0156, -0.0755,  0.2701],
        [ 0.0190,  0.0418,  0.2677]])

Size of shape 1 : torch.Size([446, 3])
Landmarks of shape 1 : tensor([[0.0314, 0.0201, 0.2025],
        [0.0319, 0.0535, 0.1964]])

Size of shape 2 : torch.Size([465, 3])
Landmarks of shape 2 : tensor([[-0.0111,  0.0480,  0.1389],
        [-0.0108, -0.0148,  0.1165]])

Size of shape 3 : torch.Size([444, 3])
Landmarks of shape 3 : tensor([[ 0.0917, -0.3005,  0.1144],
        [ 0.0360,  0.2349,  0.0129]])



In [1]:
from skshapes.morphing import RigidMotion
from skshapes.data import read, Shape
import torch

mesh = read("data/SCAPE_low_resolution/mesh000.ply")

r = RigidMotion()
parameter = torch.Tensor([[5, 5, 5], [0, 0, 0]])
newpoints = r.fit(source=mesh).morph(parameter=parameter)

mesh.to_pyvista().save("source.vtk")

newmesh = Shape(points=newpoints, faces=mesh.faces)
newmesh.to_pyvista().save("target.vtk")

In [4]:
from skshapes.data import read
from skshapes.loss import NearestNeighborsLoss, LandmarkLoss
from skshapes.morphing import RigidMotion
from skshapes.tasks import Registration
import torch


source = read("source.vtk")
target = read("target.vtk")

source.landmarks = torch.arange(source.points.shape[0], dtype=torch.int64)
target.landmarks = torch.arange(target.points.shape[0], dtype=torch.int64)

# Apply ICP
r = Registration(
    model=RigidMotion(),
    loss=LandmarkLoss(),
    verbose=1,
    n_iter=5,
    device="cuda:0",
)

morphed = r.fit_transform(source=source, target=target)
morphed.to_pyvista().save("output.vtk")

Loss value at iteration 0 : 19.28798484802246
Loss value at iteration 1 : 0.3848213851451874
Loss value at iteration 2 : 0.007739880122244358
Loss value at iteration 3 : 0.0030395323410630226
Loss value at iteration 4 : 0.0030395323410630226


In [1]:
from skshapes.data import read, Shape
import pyvista
import torch

mesh = pyvista.read("output.vtk")

if mesh.is_all_triangles():

    triangles = mesh.faces.reshape(-1, 4)[:, 1:].T
    triangles = torch.from_numpy(triangles).long()
    shape = Shape(points=torch.from_numpy(mesh.points), triangles=triangles)

In [2]:
import pyvista.examples

In [6]:
import numpy as np


def is_surface(mesh):
    """Test if the mesh represents a surface. This is done by checking if the

    Args:
        mesh (_type_): _description_

    Returns:
        _type_: _description_
    """
    return mesh.triangulate().is_all_triangles()


def is_triangular_surface(mesh):
    return mesh.is_all_triangles()


def is_1d_complex(mesh):

    if is_surface(mesh):
        return False

    lines = mesh.lines
    if len(lines) == 0:
        return False

    if len(lines) % 3 != 0:
        return False

    return np.all(lines.reshape(-1, 3)[:, 0] == 2)


cow = pyvista.examples.download_cow()
triangulated_cow = cow.triangulate()
skeleton_cow = cow.extract_all_edges()

weird_cow = triangulated_cow.copy()

# weird_cow.faces = np.array([], dtype=np.int64)
weird_cow.lines = skeleton_cow.lines

print(
    "Is cow a surface? {}\n  a triangular surface? {}".format(
        is_surface(cow), is_triangular_surface(cow)
    )
)
print("  a 1d complex? {}".format(is_1d_complex(cow)))

print(
    "Is triangulated cow a surface? {}\n  a triangular surface? {}".format(
        is_surface(triangulated_cow), is_triangular_surface(triangulated_cow)
    )
)
print("  a 1d complex? {}".format(is_1d_complex(triangulated_cow)))

print(
    "Is skeleton cow a surface? {}\n  a triangular surface? {}".format(
        is_surface(skeleton_cow), is_triangular_surface(skeleton_cow)
    )
)
print("  a 1d complex? {}".format(is_1d_complex(skeleton_cow)))

print(
    "Is weird cow a surface? {}\n  a triangular surface? {}".format(
        is_surface(weird_cow), is_triangular_surface(weird_cow)
    )
)
print("  a 1d complex? {}".format(is_1d_complex(weird_cow)))

Is cow a surface? True
  a triangular surface? False
  a 1d complex? False
Is triangulated cow a surface? True
  a triangular surface? True
  a 1d complex? False
Is skeleton cow a surface? False
  a triangular surface? False
  a 1d complex? True
Is weird cow a surface? True
  a triangular surface? False
  a 1d complex? False


In [9]:
import pyvista
import pyvista.examples
from skshapes.data import Shape
import torch

mesh = pyvista.read("data/SCAPE_low_resolution/mesh000.ply")
mesh = pyvista.examples.download_cow()

if is_triangular_surface(mesh):
    triangles = mesh.faces.reshape(-1, 4)[:, 1:].T
    shape = Shape(
        points=torch.from_numpy(mesh.points),
        triangles=torch.from_numpy(triangles).long(),
    )

if is_surface(mesh):
    triangles = mesh.triangulate().faces.reshape(-1, 4)[:, 1:].T
    shape = Shape(
        points=torch.from_numpy(mesh.points),
        triangles=torch.from_numpy(triangles).long(),
    )

In [10]:
lines = skeleton_cow.lines

lines = np.concatenate([lines, np.zeros(2, dtype=np.int64)], axis=0)

lines.reshape(-1, 3)[:, 0:1].T

ValueError: cannot reshape array of size 18497 into shape (3)

In [66]:
mesh.verts

array([], dtype=int64)

In [2]:
from skshapes.data import read, Shape
import torch

shape = read("data/SCAPE_low_resolution/mesh000.ply")

# new_shape = Shape(points=points, triangles=triangles)
# new_shape.to_pyvista().save("triangular_surface.vtk")

print(shape.device)
print(shape.triangles.device)
print(shape.to("cuda:0").device)
print(shape.device)
shape.device = torch.device("cuda:0")
print(shape.device)
print(shape.triangles.device)

cpu
cpu
cuda:0
cpu
cuda:0
cuda:0


In [6]:
print("Number of points: {}".format(shape.npoints))
print("Number of edges: {}".format(shape.nedges))
print("Number of triangles: {}".format(shape.ntriangles))

print(shape.ntriangles)

print("Number of points: {}".format(shape.npoints))
print("Number of edges: {}".format(shape.nedges))
print("Number of triangles: {}".format(shape.ntriangles))

Number of points: 127
Number of edges: 0
Number of triangles: 250
250
Number of points: 127
Number of edges: 0
Number of triangles: 250


In [38]:
image = pyvista.read("Sans titre.jpeg")
print("Image is a UniformGrid: ", isinstance(image, pyvista.UniformGrid))
print("Image is a PolyData: ", isinstance(image, pyvista.PolyData))

mesh = pyvista.read("data/SCAPE_low_resolution/mesh000.ply")
print("Mesh is a UniformGrid: ", isinstance(mesh, pyvista.UniformGrid))
print("Mesh is a PolyData: ", isinstance(mesh, pyvista.PolyData))

Image is a UniformGrid:  True
Image is a PolyData:  False
Mesh is a UniformGrid:  False
Mesh is a PolyData:  True


In [28]:
new_shape.edges
new_shape._triangles = None

new_shape.to_pyvista().save("surface.vtk")